# Treinamento word2vec

**Base dados**

- http://www.nilc.icmc.usp.br/embeddings

In [1]:
import unicodedata
import sys
import string
import time
import csv
import nltk
import re

import numpy as np
import pandas as pd
import seaborn as sns
import warnings

import nltk
import re
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
nltk.download("stopwords")
nltk.download('punkt')

from pylab import rcParams
import matplotlib.pyplot as plt
import matplotlib as m
import matplotlib as mpl

warnings.filterwarnings("ignore")

plt.style.use('fivethirtyeight')
m.rcParams['axes.labelsize'] = 14
m.rcParams['xtick.labelsize'] = 12
m.rcParams['ytick.labelsize'] = 12
m.rcParams['text.color'] = 'k'
rcParams['figure.figsize'] = 18, 8

%reload_ext watermark
%watermark -a "Rafael Gallo" --iversions

from platform import python_version
print('Versão Jupyter Notebook neste projeto:', python_version())

Author: Rafael Gallo

sys       : 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
nltk      : 3.7
pandas    : 1.4.4
spacy     : 3.5.0
matplotlib: 3.5.2
csv       : 1.0
seaborn   : 0.11.2
re        : 2.2.1
numpy     : 1.21.5

Versão Jupyter Notebook neste projeto: 3.9.13


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\55119\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\55119\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\55119\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Baixando modelo pré-treino
!python -m spacy download pt_core_news_sm

     --------------------------------------- 13.0/13.0 MB 40.9 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [3]:
# Importando modelo pré-treinado
nlp = spacy.load("pt_core_news_sm")
nlp

# Base dados

In [4]:
data1 = pd.read_csv("data/treino.csv")

In [5]:
data2 = pd.read_csv("data/teste.csv")

In [6]:
# Visualizando 10 primeiros dados
data1.head(10)

,title,text,date,category,subcategory,link
0,"Após polêmica, Marine Le Pen diz que abomina n...",A candidata da direita nacionalista à Presidên...,2017-04-28,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
1,"Macron e Le Pen vão ao 2º turno na França, em ...",O centrista independente Emmanuel Macron e a d...,2017-04-23,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
2,"Apesar de larga vitória nas legislativas, Macr...",As eleições legislativas deste domingo (19) na...,2017-06-19,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/06/189...
3,"Governo antecipa balanço, e Alckmin anuncia qu...",O número de ocorrências de homicídios dolosos ...,2015-07-24,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/07...
4,"Após queda em maio, a atividade econômica sobe...","A economia cresceu 0,25% no segundo trimestre,...",2017-08-17,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...
5,Barcelona vence de virada; Atlético de Madri b...,Depois de golear o Celtic por 7 a 0 em sua est...,2016-09-28,esporte,NaN,http://www1.folha.uol.com.br/esporte/2016/09/1...
6,'Spartacus' oferece um duplo retrato de batalh...,"""Spartacus"" (1960, TC Cult, 22h) narra a histó...",2016-09-12,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2016/12...
7,Sobe para 86 o número de mortos no atentado te...,"Um homem que ficou ferido no atentado de Nice,...",2016-08-19,mundo,NaN,http://www1.folha.uol.com.br/mundo/2016/08/180...
8,"Premiada em Sundance, Crystal Moselle retrata ...",Vencedora do prêmio do júri no Festival de Sun...,2016-11-09,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2016/09...
9,Metroviários e ferroviários ameaçam parar na p...,Funcionários do Metrô e da CPTM podem cruzar o...,2017-07-27,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2017/07...


In [7]:
# Visualizando 10 últimos dados
data1.tail()

,title,text,date,category,subcategory,link
89995,"Mural: Há 30 anos, aeroporto não foi bem receb...",Década de 1970. Congonhas já estava superlotad...,2015-01-22,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/01...
89996,"As notícias sobre Schumacher não são boas, diz...",O ex-presidente da Ferrari Luca di Montezemolo...,2016-04-02,esporte,NaN,http://www1.folha.uol.com.br/esporte/2016/02/1...
89997,"De olho em R$ 50 bilhões, governo pode concede...","Para fazer caixa, o governo estuda conceder pa...",2017-08-29,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...
89998,Moro deu a Lula o papel de coitadinho,Realizou-se parcialmente o primeiro objetivo d...,2016-06-03,colunas,eliogaspari,http://www1.folha.uol.com.br/colunas/eliogaspa...
89999,Velocidade da aprovação das reformas tem 'exce...,O Banco Central afirmou que a velocidade da ap...,2016-10-25,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/10/1...


In [8]:
# Linhas colunas
data1.shape

(90000, 6)

In [9]:
# Info dados
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        90000 non-null  object
 1   text         90000 non-null  object
 2   date         90000 non-null  object
 3   category     90000 non-null  object
 4   subcategory  17175 non-null  object
 5   link         90000 non-null  object
dtypes: object(6)
memory usage: 4.1+ MB


In [10]:
# Tipo dados
data1.dtypes

title          object
text           object
date           object
category       object
subcategory    object
link           object
dtype: object

In [11]:
# Amostra dados
data1.text.sample(150)

18012    Graças a uma série de coincidências, a jornali...
67344    Com pessoas e empresas recorrendo crescentemen...
23583    BRASÍLIA - O executivo Hilberto Mascarenhas co...
45669    O Grêmio Recreativo e Escola de Samba Mancha V...
61881    A missa de um ano da morte do cantor Cristiano...
38766    Pouco antes da Copa América, circulou pelas re...
57449    Pai e filho orientais foram esfaqueados durant...
67201    "Nós nunca vamos ter estabilidade se tivermos ...
76806    Nas bancas a partir de domingo (2), o 23º volu...
80491    Revelada pela segunda edição do "MasterChef Br...
25010    Empresas de aplicativo de transporte, como o U...
54644    No centro histórico de Tallinn, matrioskas do ...
53076    O governo do Irã fechou uma publicação semanal...
27148    Mick Jagger, vocalista dos Rolling Stones, diz...
82753    Um temporal no final da tarde desta quarta-fei...
52686    O técnico da seleção brasileira, Dunga, cumpri...
70417    Depois de grandes veículos como "The New York .

In [12]:
texto = "Rio de Janeiro é uma cidade maravilhosa"
doc = nlp(texto)
doc

Rio de Janeiro é uma cidade maravilhosa

In [13]:
textos_para_tratamento = (titulos.lower() for titulos in data1["title"])
textos_para_tratamento

<generator object <genexpr> at 0x0000013EDEC13970>

In [14]:
def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)

    if len(tokens_validos) > 2:
        return  " ".join(tokens_validos)

texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(texto)
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [15]:
texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(texto)
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [16]:
from time import time

t0 = time()
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                        batch_size = 1000,
                                                        n_process = -1)]

tf = time() - t0

titulos_tratados = pd.DataFrame({"titulo": textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


# Modelo word2vec

In [17]:
#size = 300

from gensim.models import Word2Vec

model_word2vec = Word2Vec(sg = 0,
                          window = 2,
                          min_count = 5,
                          alpha = 0.03,
                          min_alpha = 0.007)

In [18]:
print(len(titulos_tratados))

90000


In [19]:
titulos_tratados = titulos_tratados.dropna().drop_duplicates()
print(len(titulos_tratados))

84466


In [20]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]
lista_lista_tokens

[['polêmica', 'marine', 'le', 'pen', 'abomina', 'negacionistas', 'holocausto'],
 ['macron', 'le', 'pen', 'turno', 'frança', 'revés', 'siglas', 'tradicionais'],
 ['apesar',
  'larga',
  'vitória',
  'legislativas',
  'macron',
  'terá',
  'desafios',
  'frente'],
 ['governo',
  'antecipa',
  'balanço',
  'alckmin',
  'anuncia',
  'queda',
  'homicídios',
  'sp'],
 ['queda', 'maio', 'atividade', 'econômica', 'sobe', 'junho', 'bc'],
 ['barcelona',
  'vence',
  'virada',
  'atlético',
  'madri',
  'bate',
  'bayern',
  'munique'],
 ['spartacus', 'oferece', 'duplo', 'retrato', 'batalhas', 'perdidas'],
 ['sobe', 'mortos', 'atentado', 'terrorista', 'nice', 'frança'],
 ['premiada',
  'sundance',
  'crystal',
  'moselle',
  'retrata',
  'sexismo',
  'mundo',
  'skate'],
 ['metroviários', 'ferroviários', 'ameaçam', 'parar', 'terça', 'paulo'],
 ['anos', 'angeli', 'tirinhas', 'diárias', 'ilustrada'],
 ['mortes', 'acidentes', 'trânsito', 'caem', 'cidade', 'sp'],
 ['daniel', 'craig', 'será', 'stormt

# Treinamento modelo word2vec

In [21]:
import logging

# Treinamento das palavras
logging.basicConfig(format="%(asctime)s : - %(message)s", level = logging.INFO)

# Modelo 
model_word2vec2 = Word2Vec(sg = 0,
                           window = 2,
                           min_count = 5,
                           alpha = 0.03,
                           min_alpha = 0.007)

# Visualizando modelo
model_word2vec2.build_vocab(lista_lista_tokens, progress_per=5000)

2023-03-01 23:20:57,356 : - Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=100, alpha=0.03)', 'datetime': '2023-03-01T23:20:57.356885', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2023-03-01 23:20:57,357 : - collecting all words and their counts
2023-03-01 23:20:57,358 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-03-01 23:20:57,369 : - PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2023-03-01 23:20:57,378 : - PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2023-03-01 23:20:57,387 : - PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2023-03-01 23:20:57,397 : - PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2023-03-01 23:20:57,408 : - PROGRESS: at sentence #25000, processed 159589 words, keeping 23491 word t

In [22]:
# Modelo 02 - treinamento word2vec

model_word2vec2.train(lista_lista_tokens, 
                 total_examples=model_word2vec2.corpus_count,
                 epochs = 30)

2023-03-01 23:20:57,824 : - Word2Vec lifecycle event {'msg': 'training model with 3 workers on 12924 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=2 shrink_windows=True', 'datetime': '2023-03-01T23:20:57.824906', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'train'}
2023-03-01 23:20:58,160 : - worker thread finished; awaiting finish of 2 more threads
2023-03-01 23:20:58,163 : - worker thread finished; awaiting finish of 1 more threads
2023-03-01 23:20:58,166 : - worker thread finished; awaiting finish of 0 more threads
2023-03-01 23:20:58,167 : - EPOCH - 1 : training on 540242 raw words (486170 effective words) took 0.3s, 1484360 effective words/s
2023-03-01 23:20:58,488 : - worker thread finished; awaiting finish of 2 more threads
2023-03-01 23:20:58,492 : - worker thread finished; awaiting finish of 1 more threads
2023-03-01 23:20:58,496 : - worker threa

(14584344, 16207260)

In [23]:
# Visualizando os textos similar 1
model_word2vec2.wv.most_similar("google")

[('facebook', 0.6431851983070374),
 ('apple', 0.6156241297721863),
 ('amazon', 0.5430499315261841),
 ('airbnb', 0.5387973189353943),
 ('waze', 0.5111874341964722),
 ('uber', 0.4895358681678772),
 ('tesla', 0.4723462462425232),
 ('yahoo', 0.45732444524765015),
 ('canais', 0.4552015960216522),
 ('news', 0.45471805334091187)]

In [24]:
# Visualizando os textos similar 2
model_word2vec2.wv.most_similar("microsoft")

[('amazon', 0.5757077932357788),
 ('unilever', 0.5674010515213013),
 ('sky', 0.5199122428894043),
 ('moma', 0.5175716876983643),
 ('twitter', 0.5173408389091492),
 ('canais', 0.5161731839179993),
 ('sony', 0.5154306292533875),
 ('buffett', 0.513797402381897),
 ('editora', 0.5131805539131165),
 ('tesla', 0.5128102898597717)]

In [25]:
# Visualizando os textos similar 3
model_word2vec2.wv.most_similar("barcelona")

[('barça', 0.6452637910842896),
 ('madrid', 0.6277923583984375),
 ('chelsea', 0.6215014457702637),
 ('psg', 0.6169667840003967),
 ('united', 0.5940908193588257),
 ('lazio', 0.5883967876434326),
 ('bayern', 0.5855112075805664),
 ('figueirense', 0.5820850133895874),
 ('juventus', 0.5798176527023315),
 ('liverpool', 0.5728926658630371)]

In [26]:
# # Visualizando os textos similar 4
model_word2vec2.wv.most_similar("messi")

[('suárez', 0.6392433643341064),
 ('cristiano', 0.6237834095954895),
 ('cavani', 0.6094532608985901),
 ('benzema', 0.573591411113739),
 ('barcelona', 0.5504539012908936),
 ('tevez', 0.5500736832618713),
 ('chuteiras', 0.5437516570091248),
 ('barça', 0.5358842611312866),
 ('ronaldo', 0.5310251712799072),
 ('neymar', 0.5221753120422363)]

In [27]:
# Visualizando os textos similar 5
model_word2vec2.wv.most_similar("gm")

[('embraer', 0.672080934047699),
 ('chrysler', 0.6674906015396118),
 ('braskem', 0.6476341485977173),
 ('toyota', 0.6289199590682983),
 ('honda', 0.6241251826286316),
 ('renault', 0.6209695339202881),
 ('volks', 0.6142136454582214),
 ('inbev', 0.5989470481872559),
 ('fiat', 0.5982017517089844),
 ('tesla', 0.5909039378166199)]

# Treinamento word2vec -  Skip-Gram

In [31]:
#Treinamento do modelo Skip-Gram
w2v_modelo_sg = Word2Vec(sg = 1,
                      window = 5,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

# Vocabularios modelos
w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per=5000)

# Treinamento modelo
w2v_modelo_sg.train(lista_lista_tokens, 
                 total_examples=w2v_modelo_sg.corpus_count,
                 epochs = 100)

2023-03-01 23:23:00,657 : - Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=100, alpha=0.03)', 'datetime': '2023-03-01T23:23:00.657626', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2023-03-01 23:23:00,658 : - collecting all words and their counts
2023-03-01 23:23:00,659 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-03-01 23:23:00,670 : - PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2023-03-01 23:23:00,681 : - PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2023-03-01 23:23:00,691 : - PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2023-03-01 23:23:00,709 : - PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2023-03-01 23:23:00,722 : - PROGRESS: at sentence #25000, processed 159589 words, keeping 23491 word t

(14584246, 16207260)

In [32]:
# Visualizando os textos similar
w2v_modelo_sg.wv.most_similar("google")

[('facebook', 0.6943855285644531),
 ('yahoo', 0.6384097337722778),
 ('apple', 0.6355850696563721),
 ('android', 0.6233773827552795),
 ('waze', 0.5991013646125793),
 ('chips', 0.5856183171272278),
 ('reguladores', 0.577750027179718),
 ('verizon', 0.554513692855835),
 ('toshiba', 0.5517430305480957),
 ('app', 0.5492886900901794)]

In [39]:
# Visualizando os textos similar
w2v_modelo_sg.wv.most_similar("microsoft")

[('linkedin', 0.5810561180114746),
 ('chips', 0.5792808532714844),
 ('kraft', 0.540615975856781),
 ('verizon', 0.5347290635108948),
 ('monsanto', 0.5258455872535706),
 ('telefónica', 0.5249852538108826),
 ('alibaba', 0.514167845249176),
 ('ciberataques', 0.5134427547454834),
 ('unilever', 0.5125748515129089),
 ('sabmiller', 0.5047908425331116)]

In [43]:
# Visualizando os textos similar
w2v_modelo_sg.wv.most_similar("coca")

[('cultivo', 0.6734629273414612),
 ('medicinal', 0.5522952675819397),
 ('disponível', 0.5265748500823975),
 ('comemorativa', 0.5007907748222351),
 ('conversão', 0.48705345392227173),
 ('multinacional', 0.4865379333496094),
 ('vacinas', 0.48320677876472473),
 ('pesquisadores', 0.4818926751613617),
 ('deslocados', 0.4807136058807373),
 ('refrigerantes', 0.4793591797351837)]

In [33]:
# Visualizando os textos similar
w2v_modelo_sg.wv.most_similar("gm")

[('metalúrgicos', 0.6612794399261475),
 ('bmw', 0.6585705876350403),
 ('fiat', 0.6326159834861755),
 ('motors', 0.62115478515625),
 ('honda', 0.5904354453086853),
 ('audi', 0.5885869264602661),
 ('montadora', 0.5882266759872437),
 ('chrysler', 0.583420991897583),
 ('volks', 0.5817675590515137),
 ('autônomos', 0.5771198868751526)]

In [35]:
# Visualizando os textos similar
w2v_modelo_sg.wv.most_similar("bmw")

[('gm', 0.6585706472396851),
 ('tesla', 0.6290761828422546),
 ('toyota', 0.623538613319397),
 ('reestrutura', 0.6203257441520691),
 ('audi', 0.6162607073783875),
 ('apreender', 0.5914158225059509),
 ('chrysler', 0.575086772441864),
 ('honda', 0.5682542324066162),
 ('airbags', 0.5656874179840088),
 ('reguladores', 0.551045298576355)]

# Salvando modelos

In [37]:
model_word2vec.wv.save_word2vec_format("data/modelo_cbow.txt", binary=False)

2023-03-01 23:25:37,285 : - attribute count not present in <gensim.models.keyedvectors.KeyedVectors object at 0x0000013ED93626D0>; will store in internal index_to_key order
2023-03-01 23:25:37,286 : - storing 0x100 projection weights into data/modelo_cbow.txt


In [38]:
w2v_modelo_sg.wv.save_word2vec_format("data/modelo_skipgram.txt", binary=False)

2023-03-01 23:25:41,407 : - storing 12924x100 projection weights into data/modelo_skipgram.txt
